In [0]:
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/
!git checkout lmdb

Cloning into 'Generative_Models'...
remote: Enumerating objects: 2077, done.
remote: Counting objects: 100% (2077/2077), done.
remote: Compressing objects: 100% (592/592), done.
remote: Total 14901 (delta 1504), reused 1906 (delta 1367), pack-reused 12824
Receiving objects: 100% (14901/14901), 414.67 MiB | 36.14 MiB/s, done.
Resolving deltas: 100% (10105/10105), done.
Checking out files: 100% (1675/1675), done.
/content/Generative_Models
Branch 'lmdb' set up to track remote branch 'lmdb' from 'origin'.
Switched to a new branch 'lmdb'


In [0]:
!pip -q install keras
!pip -q install colorlog
!pip -q install keras-radam

# Settings

In [0]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
print(sep_local)

env: TF_KERAS=1
/


In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [0]:
os.chdir('/content/Generative_Models/')
print(os.getcwd())

/content/Generative_Models


# Dataset loading

In [0]:
dataset_name='pokemon'

In [0]:
images_dir = '/content/Generative_Models/data/.pokemon/'
validation_percentage = 0
valid_format = 'png'

In [0]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [0]:
!ls -la /content/Generative_Models/data/.pokemon

total 28
drwxr-xr-x 3 root root  4096 Jun 14 12:09 .
drwxr-xr-x 6 root root  4096 Jun 14 12:09 ..
drwxr-xr-x 2 root root 20480 Jun 14 12:09 DS06


In [0]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format,
    verbose = 1
)

  DEBUG    | Looking for images in 'DS06'
  INFO     | 809 file found


In [0]:
import numpy as np
import lmdb
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img

In [0]:
from utils.data_and_files.file_utils import get_file_path

In [0]:
def store_lmdb(
             lmdb_name,
             image_lists,
             category, 
             image_dir,
             target_size=None,
             color_mode='rgb',
             save_to_dir=None,
             save_prefix='',
             save_format='png'
):

    classes = list(image_lists.keys())
    num_class = len(classes)

    class2id = dict(zip(classes, range(len(classes))))
    id2class = dict((v, k) for k, v in class2id.items())

    if color_mode not in {'rgb', 'grayscale'}:
        raise ValueError('Invalid color mode:', color_mode, '; expected "rgb" or "grayscale".')

    lmdb_env = lmdb.open(lmdb_name)
    # lmdb_db = lmdb_env.begin(write=True)
    # print(classes)
    with lmdb_env.begin(write=True) as lmdb_db:
      for label_name in classes:
          for j, _ in enumerate(image_lists[label_name][category]):
              img_path = get_file_path(image_lists,
                                        label_name,
                                        j,
                                        image_dir,
                                        category)
              img = img_to_array(
                              load_img(
                              img_path,
                              grayscale=color_mode=='grayscale',
                              target_size=target_size
                              )
                          )
              lmdb_db.put(label_name.encode(), img, append=True, overwrite=False)      
    lmdb_env.close()

In [0]:
store_lmdb(lmdb_name='pokemon2', image_lists=imgs_list, category='training', image_dir=images_dir)

In [0]:
lmdb_env = lmdb.open('pokemon2')
with lmdb_env.begin() as lmdb_txn:
    with lmdb_txn.cursor() as lmdb_cursor:
        for key, value in lmdb_cursor:  
           print (key)
           print (value)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
keys = [key for key, _ in lmdb_db.cursor() ]
values = [value for _, value in lmdb_db.cursor() ]

NameError: ignored

In [0]:
print(np.fromstring(values))

In [0]:
import numpy as np
import lmdb

def write_lmdb(filename):
    print ('Write lmdb')

    lmdb_env = lmdb.open(filename, map_size=int(1e9))

    n_samples= 2
    X= (255*np.random.rand(n_samples,3,4,3)).astype(np.uint8)
    y= np.random.rand(n_samples).astype(np.float32)

    for i in range(n_samples):
        with lmdb_env.begin(write=True) as lmdb_txn:
            lmdb_txn.put('X_'+str(i), X)
            lmdb_txn.put('y_'+str(i), y)

            print ('X:',X)
            print ('y:',y)

def read_lmdb(filename):
    print ('Read lmdb')

    lmdb_env = lmdb.open(filename)
    lmdb_txn = lmdb_env.begin()
    lmdb_cursor = lmdb_txn.cursor()

    n_samples=0
    with lmdb_env.begin() as lmdb_txn:
        with lmdb_txn.cursor() as lmdb_cursor:
            for key, value in lmdb_cursor:  
                print (key)
                if('X' in key):
                    print (np.fromstring(value, dtype=np.uint8))
                if('y' in key):
                    print (np.fromstring(value, dtype=np.float32))

                n_samples=n_samples+1

    print ('n_samples',n_samples)

write_lmdb('temp.db')
read_lmdb('temp.db')